In [3]:
!pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.3 MB/s eta 0:00:0

In [5]:
import fiftyone as fo
import fiftyone.zoo as foz

classes = ["cat", "bicycle", "backpack", "potted plant"]

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["segmentations"],
    classes=classes,
    max_samples=5000
)

dataset.export(
    export_dir="coco_subset",
    dataset_type=fo.types.COCODetectionDataset,
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Downloading 5000 images


 100% |████████████████| 5000/5000 [18.0m elapsed, 0s remaining, 4.4 images/s]      


...

INFO:eta.core.utils: 100% |███████████████| 5000/5000 [2.1m elapsed, 0s remaining, 28.8 samples/s]      


Dataset 'coco-2017-train-5000' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-train-5000' created


 100% |███████████████| 5000/5000 [9.3m elapsed, 0s remaining, 9.8 samples/s]       


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [9.3m elapsed, 0s remaining, 9.8 samples/s]       


In [6]:
import cv2
import numpy as np
from pycocotools.coco import COCO
import os

coco = COCO('annotations/instances_train2017.json')

categories = ['cat', 'bicycle', 'backpack', 'tree']
cat_ids = coco.getCatIds(catNms=categories)
img_ids = coco.getImgIds(catIds=cat_ids)

for img_id in img_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join('train2017', img_info['file_name'])
    img = cv2.imread(img_path)
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(ann_ids)

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    for ann in anns:
        m = coco.annToMask(ann)
        mask = np.maximum(mask, m)

    mask = cv2.resize(mask, (128, 128)) * 255
    cv2.imwrite(f'silhouettes/{img_info["file_name"]}', mask)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'annotations/instances_train2017.json'